In [19]:
pip install -r ../../requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [20]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [21]:
%run -i ../../zkstats/core.py

In [22]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
comb_data_path = os.path.join('prover/comb_data.json')

=======================  ZK-STATS FLOW =======================

In [23]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

f_raw_input = open(data_path, "r")
data = json.loads(f_raw_input.read())["input_data"][0]
data_tensor = torch.reshape(torch.tensor(data),(1, len(data), 1))

#  dummy data for data consumer: arbitraryyy, just to make sure after filtered, it's not empty
dummy_data = np.random.uniform(1, 100, len(data))
json.dump({"input_data":[dummy_data.tolist()]}, open(dummy_data_path, 'w'))

# where(element > 30)
dummy_data_tensor = torch.reshape(torch.tensor(dummy_data), (1, len(dummy_data),1 ))
gt30_dummy_data_tensor = dummy_data_tensor[dummy_data_tensor > 30].reshape(1,-1,1)
dummy_theory_output = torch.exp(torch.mean(torch.log(gt30_dummy_data_tensor)))



In [24]:
# Verifier/ data consumer side: send desired calculation
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = dummy_theory_output, requires_grad = False)

    def forward(self,X):
        # where part
        num_fil_X = torch.sum((X>30).double())
        fil_X = torch.where(X>30, X, 1.0)
        return (torch.abs((torch.log(self.w)*num_fil_X)-torch.sum(torch.log(fil_X)))<=num_fil_X*torch.log(torch.tensor(1.01)), self.w)
    
verifier_define_calculation(verifier_model, verifier_model_path, [dummy_data_path])

/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_88041/2680539101.py:12: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  return (torch.abs((torch.log(self.w)*num_fil_X)-torch.sum(torch.log(fil_X)))<=num_fil_X*torch.log(torch.tensor(1.01)), self.w)


In [25]:
# prover calculates settings, send to verifier
gt30_data_tensor = data_tensor[data_tensor > 30].reshape(1,-1,1)
print("new tensor: ", gt30_data_tensor.size())
theory_output = torch.exp(torch.mean(torch.log(gt30_data_tensor)))
print("Theory_output: ", theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = theory_output, requires_grad = False)
        
    def forward(self,X):
        # where part
        num_fil_X = torch.sum((X>30).double())
        fil_X = torch.where(X>30, X, 1.0)

        return (torch.abs((torch.log(self.w)*num_fil_X)-torch.sum(torch.log(fil_X)))<=num_fil_X*torch.log(torch.tensor(1.01)), self.w)

    

prover_gen_settings([data_path], comb_data_path, prover_model,prover_model_path,[8], "resources", settings_path)

/var/folders/89/y9dw12v976ngdmqz4l7wbsnr0000gn/T/ipykernel_88041/2135116582.py:17: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  return (torch.abs((torch.log(self.w)*num_fil_X)-torch.sum(torch.log(fil_X)))<=num_fil_X*torch.log(torch.tensor(1.01)), self.w)


new tensor:  torch.Size([1, 272, 1])
Theory_output:  tensor(50.8632)
==== Generate & Calibrate Setting ====
scale:  [8]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":8,"param_scale":8,"scale_rebase_multiplier":10,"lookup_range":[-196540,45056],"logrows":18,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":14432,"total_assignments":3615,"total_const_size":306,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,8],"model_input_scales":[8],"module_sizes":{"kzg":[],"poseidon":[14432,[1]],"elgamal":[0,[0]]},"required_lookups":["Abs",{"Div":{"denom":100.49927}},{"Ln":{"scale":256.0}},{"GreaterThan":{"a":0.0}}],"check_mode":"UNSAFE","version":"7.0.0","num_blinding_factors":null}


In [26]:
# Here verifier & prover can concurrently call setup since all params are public to get pk. 
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
verifier_setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, comb_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

==== setting up ezkl ====


spawning module 0
spawning module 2
spawning module 0
spawning module 2


Time setup: 32.721121072769165 seconds
Theory output:  tensor(50.8632)
!@# compiled_model exists? True
!@# compiled_model exists? True
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 50.86328125
==== Generating Proof ====


spawning module 0
spawning module 2


proof:  {'instances': [[[11768814371718170976, 435173728250646979, 519717007263840094, 1741290966923863957], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [9740814119635710701, 8723924064432029923, 17927155970413989335, 152971583043225146]]], 'proof': '1ea394cbd63e62f54a0735a948bf3b26255b2246af8944252cd73d747e15ca5f1a9b695b217192042eb0f11755cd54a6b37e75249a7450cee611278ad702fe201555837ec2599171957402ba432d13c58fed97b7cb98bf426dd2bb19e442a6fa07e40b6575c37dbc6d407f184904171bb0fba04710472cde28041ad196eb1057072ef2bad4b104058563724a632c359b355ab2804e46c61abe1d2edaad278cc308a3f25d984c264085a206f5cb8fd23171a8cf9a046786a127a839d6984ea07c0abf8dcb56fc67f5ed9b5f20487f652a18dbd1bc845c1ba35c23f1f3f935a3da2150e9405ea66e655faac1bba85ce9cc9af9f4c1bca9d50f8aa3811ce4d90dd52511c25a3f22c71a2e620db684851a67e0395349949d10360d2813021325781c12c500987adbf0640fed4b445e16fceb99c4fc5232fb278143e4dd260937c1140a2f2a9ffcf2072e1e8c1331f4dedadb6b2cb41a9777d3fa406ca588fd84974618

In [27]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path)

num_inputs:  1
prf instances:  [[[11768814371718170976, 435173728250646979, 519717007263840094, 1741290966923863957], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [9740814119635710701, 8723924064432029923, 17927155970413989335, 152971583043225146]]]
proof boolean:  1.0
proof result 1 : 50.86328125
verified
